<a href="https://colab.research.google.com/github/gizemnurbektas/CNN-Based-Receipt-Fraud-Detection/blob/main/CNN_Based_Receipt_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!mkdir -p /content/receipt_fraud_cnn
%cd /content/receipt_fraud_cnn
!mkdir -p data


/content/receipt_fraud_cnn


In [22]:
!unzip -q /content/receiptimages.zip -d /content/receipt_fraud_cnn/data
!find /content/receipt_fraud_cnn/data -maxdepth 3 -type f | head


/content/receipt_fraud_cnn/data/real/072.jpg
/content/receipt_fraud_cnn/data/real/058.jpg
/content/receipt_fraud_cnn/data/real/068.jpg
/content/receipt_fraud_cnn/data/real/061.jpg
/content/receipt_fraud_cnn/data/real/064.jpg
/content/receipt_fraud_cnn/data/real/112.jpg
/content/receipt_fraud_cnn/data/real/084.jpg
/content/receipt_fraud_cnn/data/real/063.jpg
/content/receipt_fraud_cnn/data/real/067.jpg
/content/receipt_fraud_cnn/data/real/086.jpg


In [23]:
!ls /content


receipt_fraud_cnn  receiptimages.zip  sample_data


In [24]:
!pip -q install gradio opencv-python


In [25]:

import torch
import torch.nn as nn
import torch.nn.functional as F

class ReceiptFraudCNN(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.drop = nn.Dropout(0.3)

        # input: (3,224,224) -> 3 kez pool -> (64,28,28)
        self.fc1 = nn.Linear(64 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = self.drop(F.relu(self.fc1(x)))
        return self.fc2(x)


In [26]:

import torch
import torch.nn as nn
import torch.nn.functional as F

class ReceiptFraudCNN(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.drop = nn.Dropout(0.3)

        # input: (3,224,224) -> 3 kez pool -> (64,28,28)
        self.fc1 = nn.Linear(64 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = self.drop(F.relu(self.fc1(x)))
        return self.fc2(x)


In [27]:
!find /content/receipt_fraud_cnn/data -maxdepth 4 -type d


/content/receipt_fraud_cnn/data
/content/receipt_fraud_cnn/data/real
/content/receipt_fraud_cnn/data/receiptimages
/content/receipt_fraud_cnn/data/fake


In [28]:
!find /content/receipt_fraud_cnn/data -maxdepth 5 -type f | head -n 20


/content/receipt_fraud_cnn/data/real/072.jpg
/content/receipt_fraud_cnn/data/real/058.jpg
/content/receipt_fraud_cnn/data/real/068.jpg
/content/receipt_fraud_cnn/data/real/061.jpg
/content/receipt_fraud_cnn/data/real/064.jpg
/content/receipt_fraud_cnn/data/real/112.jpg
/content/receipt_fraud_cnn/data/real/084.jpg
/content/receipt_fraud_cnn/data/real/063.jpg
/content/receipt_fraud_cnn/data/real/067.jpg
/content/receipt_fraud_cnn/data/real/086.jpg
/content/receipt_fraud_cnn/data/real/109.jpg
/content/receipt_fraud_cnn/data/real/103.jpg
/content/receipt_fraud_cnn/data/real/078.jpg
/content/receipt_fraud_cnn/data/real/073.jpg
/content/receipt_fraud_cnn/data/real/098.jpg
/content/receipt_fraud_cnn/data/real/107.jpg
/content/receipt_fraud_cnn/data/real/091.jpg
/content/receipt_fraud_cnn/data/real/085.jpg
/content/receipt_fraud_cnn/data/real/113.jpg
/content/receipt_fraud_cnn/data/real/106.jpg


In [30]:

import os
import glob
import random
import numpy as np
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from model import ReceiptFraudCNN

IMG_SIZE = 224
BATCH = 32
EPOCHS = 8
LR = 1e-3
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

def list_images(folder):
    exts = ["*.jpg", "*.jpeg", "*.png", "*.JPG", "*.JPEG", "*.PNG"]
    files = []
    for e in exts:
        files.extend(glob.glob(os.path.join(folder, e)))
    return files

def find_real_fake_dirs(root="/content/receipt_fraud_cnn/data"):
    real_dir, fake_dir = None, None
    for dirpath, dirnames, _ in os.walk(root):
        base = os.path.basename(dirpath).lower()
        if base == "real":
            real_dir = dirpath
        if base == "fake":
            fake_dir = dirpath
    return real_dir, fake_dir

def read_rgb(path):
    img = cv2.imread(path)
    if img is None:
        raise ValueError(f"Cannot read: {path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def augment(img):
    if random.random() < 0.5:
        angle = random.uniform(-6, 6)
        h, w = img.shape[:2]
        M = cv2.getRotationMatrix2D((w/2, h/2), angle, 1.0)
        img = cv2.warpAffine(img, M, (w, h), borderMode=cv2.BORDER_REPLICATE)

    if random.random() < 0.5:
        alpha = random.uniform(0.85, 1.15)
        beta  = random.uniform(-15, 15)
        img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

    if random.random() < 0.3:
        img = cv2.GaussianBlur(img, (3,3), 0)

    return img

def preprocess(img):
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32) / 255.0
    img = np.transpose(img, (2, 0, 1))
    return img

class RealFakeDataset(Dataset):
    def __init__(self, real_paths, fake_paths):
        self.items = [(p, 1) for p in real_paths] + [(p, 0) for p in fake_paths]
        random.shuffle(self.items)

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        path, label = self.items[idx]
        img = read_rgb(path)
        img = augment(img)
        x = preprocess(img)
        return torch.tensor(x, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

def split_items(items, val_ratio=0.2):
    n = len(items)
    n_val = int(n * val_ratio)
    return items[n_val:], items[:n_val]

def main():
    real_dir, fake_dir = find_real_fake_dirs()
    print("Found real_dir:", real_dir)
    print("Found fake_dir:", fake_dir)

    if real_dir is None or fake_dir is None:
        raise RuntimeError("real/fake klasörleri bulunamadı. unzip sonrası yolu kontrol et.")

    real_paths = list_images(real_dir)
    fake_paths = list_images(fake_dir)

    print("real files:", len(real_paths))
    print("fake files:", len(fake_paths))

    if len(real_paths) < 10 or len(fake_paths) < 10:
        raise RuntimeError(f"Yetersiz veri. real={len(real_paths)}, fake={len(fake_paths)}")

    # Dengele
    k = min(len(real_paths), len(fake_paths))
    real_paths = random.sample(real_paths, k)
    fake_paths = random.sample(fake_paths, k)

    dataset = [(p, 1) for p in real_paths] + [(p, 0) for p in fake_paths]
    random.shuffle(dataset)

    train_items, val_items = split_items(dataset, val_ratio=0.2)

    train_real = [p for p,l in train_items if l==1]
    train_fake = [p for p,l in train_items if l==0]
    val_real   = [p for p,l in val_items if l==1]
    val_fake   = [p for p,l in val_items if l==0]

    train_ds = RealFakeDataset(train_real, train_fake)
    val_ds   = RealFakeDataset(val_real, val_fake)

    train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True)
    val_loader   = DataLoader(val_ds, batch_size=BATCH, shuffle=False)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = ReceiptFraudCNN().to(device)
    opt = torch.optim.Adam(model.parameters(), lr=LR)
    loss_fn = nn.CrossEntropyLoss()

    best_acc = 0.0

    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0.0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            opt.zero_grad()
            logits = model(x)
            loss = loss_fn(logits, y)
            loss.backward()
            opt.step()
            total_loss += loss.item()

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                pred = model(x).argmax(dim=1)
                correct += (pred == y).sum().item()
                total += y.size(0)

        acc = correct / max(total, 1)
        print(f"Epoch {epoch+1}/{EPOCHS} | loss={total_loss/len(train_loader):.4f} | val_acc={acc:.3f}")

        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), "/content/receipt_fraud_cnn/cnn_model.pth")

    print("Best val acc:", best_acc)
    print("Saved: /content/receipt_fraud_cnn/cnn_model.pth")

if __name__ == "__main__":
    main()


Found real_dir: /content/receipt_fraud_cnn/data/real
Found fake_dir: /content/receipt_fraud_cnn/data/fake
real files: 62
fake files: 52
Epoch 1/8 | loss=0.7753 | val_acc=0.500
Epoch 2/8 | loss=0.7144 | val_acc=0.500
Epoch 3/8 | loss=0.6932 | val_acc=0.500
Epoch 4/8 | loss=0.6873 | val_acc=0.650
Epoch 5/8 | loss=0.6736 | val_acc=0.550
Epoch 6/8 | loss=0.6461 | val_acc=0.700
Epoch 7/8 | loss=0.5933 | val_acc=0.600
Epoch 8/8 | loss=0.5596 | val_acc=0.550
Best val acc: 0.7
Saved: /content/receipt_fraud_cnn/cnn_model.pth


In [31]:
!python /content/receipt_fraud_cnn/train.py


Found real_dir: /content/receipt_fraud_cnn/data/real
Found fake_dir: /content/receipt_fraud_cnn/data/fake
real files: 62
fake files: 52
Epoch 1/8 | loss=0.7753 | val_acc=0.500
Epoch 2/8 | loss=0.7144 | val_acc=0.500
Epoch 3/8 | loss=0.6932 | val_acc=0.500
Epoch 4/8 | loss=0.6873 | val_acc=0.650
Epoch 5/8 | loss=0.6736 | val_acc=0.550
Epoch 6/8 | loss=0.6461 | val_acc=0.700
Epoch 7/8 | loss=0.5933 | val_acc=0.600
Epoch 8/8 | loss=0.5596 | val_acc=0.550
Best val acc: 0.7
Saved: /content/receipt_fraud_cnn/cnn_model.pth


In [32]:

import os
import numpy as np
import torch
import gradio as gr
from PIL import Image
import cv2

from model import ReceiptFraudCNN

IMG_SIZE = 224
MODEL_PATH = "/content/receipt_fraud_cnn/cnn_model.pth"
LABELS = {1: "REAL (Orijinal)", 0: "FAKE (Manipülasyon Şüphesi)"}

def preprocess_pil(pil_img):
    img = np.array(pil_img.convert("RGB"))
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32) / 255.0
    img = np.transpose(img, (2, 0, 1))
    x = torch.tensor(img, dtype=torch.float32).unsqueeze(0)
    return x

if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Model yok: {MODEL_PATH}. Önce train.py çalıştır.")

model = ReceiptFraudCNN()
model.load_state_dict(torch.load(MODEL_PATH, map_location="cpu"))
model.eval()
softmax = torch.nn.Softmax(dim=1)

def predict(image):
    if image is None:
        return "Error: Görsel yükleyin.", 0.0

    x = preprocess_pil(image)
    with torch.no_grad():
        logits = model(x)
        probs = softmax(logits).numpy()[0]
        pred = int(np.argmax(probs))
        score = float(probs[pred])

    return LABELS[pred], score

demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil", label="Fiş/Dekont Görseli Yükle"),
    outputs=[gr.Textbox(label="Sonuç"), gr.Number(label="Güven Skoru")],
    title="Receipt Fraud Detector (CNN)",
    description="Real/Fake fiş görselleriyle eğitilmiş basit CNN modeli. Görsel yükle, sonucu al.",
)

if __name__ == "__main__":
    demo.launch(share=True)


Writing /content/receipt_fraud_cnn/serve.py


In [ ]:
!python /content/receipt_fraud_cnn/serve.py


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://d64622b9ed9c411fb5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
